In [46]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import log
import ultranest
from ultranest.plot import cornerplot
from scipy.stats import poisson
import pandas as pd
import healpy as hp
from healpy.newvisufunc import projview, newprojplot
import astropy.units as u
from matplotlib import colors as mcolors
from ultranest.utils import resample_equal
from scipy.interpolate import interp1d
import scipy.stats as stats
import ultranest.stepsampler
import itertools
import gc
import json
import time
from my_functions import *

In [54]:
def dipole_generate_data(Nbar_list, D_list, pathname):
    # generate data, make sure to save it for reproducibility 
    np.random.seed(42)
    NSIDE = 64
    NPIX = hp.nside2npix(NSIDE)
    dipole_theta = np.radians(48)
    dipole_phi = -np.radians(96)

    param_combinations = itertools.product(Nbar_list, D_list)
    
    for N_bar_true, D_true in param_combinations:
        m, lambda_true_array = dipole_sampling(NSIDE=64,  N_bar=N_bar_true, D=D_true,
                                                dipole_theta=dipole_theta,dipole_phi=dipole_phi)
        
        metadata = {
            'Nbar': N_bar_true,
            'D': D_true,
            'NSIDE': NSIDE,
            'NPIX': NPIX,
            'dipole_theta': dipole_theta,
            'dipole_phi': dipole_phi
        }
        
        data_to_save = {
            'm': m,
            'metadata': metadata
        }
        
        np.save(pathname+f"Datasets/Raw_Files/skymap_data_Nbar_{N_bar_true}_D_{D_true}.npy", data_to_save)
        
    return

In [53]:
def quadrupole_generate_data(Nbar_list, Q_list, pathname):
    # generate data, make sure to save it for reproducibility 
    np.random.seed(42)
    NSIDE = 64
    NPIX = hp.nside2npix(NSIDE)
    dipole_theta = np.radians(48)
    dipole_phi = -np.radians(96)
    dipole_vector = [dipole_theta, dipole_phi]
    quad_vector_1 = [np.pi/4, np.pi*4/5]
    quad_vector_2 = [np.pi*3/4 , -np.pi/4]

    param_combinations = itertools.product(Nbar_list, Q_list)
    

    for N_bar_true, Q_true in param_combinations:
        m, lambda_true_array = quadrupole_sampling(NSIDE=64,  N_bar=N_bar_true, Q=Q_true,
                                                vector_1=quad_vector_1, vector_2=quad_vector_2)
        
        metadata = {
            'Nbar': N_bar_true,
            'Q': Q_true,
            'NSIDE': NSIDE,
            'NPIX': NPIX,
            'quad_vector_1': quad_vector_1,
            'quad_vector_2': quad_vector_2
        }
        
        data_to_save = {
            'm': m,
            'metadata': metadata
        }
        
        np.save(pathname+f"Datasets/Raw_Files/skymap_data_Nbar_{N_bar_true}_Q_{Q_true}.npy", data_to_save)
        
    return

In [92]:
def dipole_quadrupole_generate_data(Nbar_list, D_list, Q_list, pathname):
    # generate data, make sure to save it for reproducibility 
    np.random.seed(42)
    NSIDE = 64
    NPIX = hp.nside2npix(NSIDE)
    dipole_theta = np.radians(48)
    dipole_phi = -np.radians(96)
    dipole_vector = [dipole_theta, dipole_phi]
    quad_vector_1 = [np.pi/4, np.pi*4/5]
    quad_vector_2 = [np.pi*3/4 , -np.pi/4]

    param_combinations = itertools.product(Nbar_list, D_list, Q_list)
    

    for N_bar_true, D_true, Q_true in param_combinations:
        m, lambda_true_array = dipole_quad_sampling(NSIDE=64,  N_bar = N_bar_true, D = D_true, Q = Q_true, d_vector=dipole_vector,
                                                q_vector_1=quad_vector_1, q_vector_2=quad_vector_2)
        
        metadata = {
            'Nbar': N_bar_true,
            'D,': D_true,
            'Q': Q_true,
            'NSIDE': NSIDE,
            'NPIX': NPIX,
            'dipole_vector': dipole_vector,
            'quad_vector_1': quad_vector_1,
            'quad_vector_2': quad_vector_2
        }
        
        data_to_save = {
            'm': m,
            'metadata': metadata
        }
        
        np.save(pathname+f"Datasets/Raw_Files/skymap_data_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}.npy", data_to_save)
        
    return

In [57]:
Nbar_list = [0.1,0.5,1,5,10,25,50,75,100,150]
D_list = [0.005,0.01,0.02,0.05,0.075,0.1]

pathname='../log_dir/Dipole_Data/'
dipole_generate_data(Nbar_list, D_list, pathname)

In [58]:
Nbar_list = [0.1,0.5,1,5,10,25,50,75,100,150]
Q_list = [0.005,0.01,0.02,0.05,0.075,0.1,0.12,0.15,0.175,0.2]

pathname='../log_dir/Quadrupole_Data/'
quadrupole_generate_data(Nbar_list, Q_list, pathname)

In [93]:
Nbar_list = [0.1,0.5,1,5,10,25,50,75,100,150]
D_list = [0.005,0.01,0.02,0.05,0.075,0.1]
Q_list = [0.005,0.01,0.02,0.05,0.075,0.1,0.12,0.15,0.175,0.2]

pathname='../log_dir/Dipole_and_Quadrupole_Data/'
dipole_quadrupole_generate_data(Nbar_list, D_list, Q_list, pathname)

In [34]:
# Create dipole skyamps
pathname='../log_dir/Dipole_Data/'
param_combinations = itertools.product(Nbar_list, D_list)
for N_bar_true, D_true in param_combinations:
    data = np.load(pathname+f"Datasets/Raw_Files/skymap_data_Nbar_{N_bar_true}_D_{D_true}.npy", allow_pickle=True).item()
    m = data['m']
    metadata = data['metadata']
    NSIDE = metadata['NSIDE']
    NPIX = metadata['NPIX']
    dipole_theta = metadata['dipole_theta']
    dipole_phi = metadata['dipole_phi']

    # Create a dipole skyamp
    dipole_plotting(m, dipole_theta, dipole_phi, cmap='plasma')
    plt.savefig(pathname+f'Datasets/Skymaps/Unsmoothed/skymap_Nbar_{N_bar_true}_D_{D_true}.png', dpi=300)

    smooth_map = compute_smooth_map(m, angle_scale=1)

    dipole_plotting(smooth_map, dipole_theta, dipole_phi, cmap='plasma')
    plt.savefig(pathname+f'Datasets/Skymaps/Smoothed/smoothed_skymap_Nbar_{N_bar_true}_D_{D_true}.png', dpi=300)
    plt.close()

In [67]:
# Create quadrupole skymaps
pathname='../log_dir/Quadrupole_Data/'
param_combinations = itertools.product(Nbar_list, Q_list)
for N_bar_true, Q_true in param_combinations:
    data = np.load(pathname+f'Datasets/Raw_files/skymap_data_Nbar_{N_bar_true}_Q_{Q_true}.npy',
                    allow_pickle=True).item()
    m = data['m']
    metadata = data['metadata']
    NSIDE = metadata['NSIDE']
    NPIX = metadata['NPIX']
    q_vector_1 = metadata['quad_vector_1']
    q_vector_2 = metadata['quad_vector_2']

    quadrupole_plotting(m, vector_1=q_vector_1, vector_2=q_vector_2, cmap='plasma')
    plt.savefig(pathname+f'Datasets/Skymaps/Unsmoothed/skymap_Nbar_{N_bar_true}_Q_{Q_true}.png', dpi=300)

    smooth_map = compute_smooth_map(m, angle_scale=1)

    quadrupole_plotting(smooth_map, vector_1=q_vector_1, vector_2=q_vector_2, cmap='plasma')
    plt.savefig(pathname+f'Datasets/Skymaps/Smoothed/smoothed_skymap_Nbar_{N_bar_true}_Q_{Q_true}.png', dpi=300)
    plt.close()

/Users/camerondavidson/opt/anaconda3/envs/dalyell_showcase/lib/python3.10/site-packages/healpy/newvisufunc.py:287: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(


In [95]:
# Create dipole+ quadrupole skymaps
pathname='../log_dir/Dipole_and_Quadrupole_Data/'
param_combinations = itertools.product(Nbar_list, D_list, Q_list)
for N_bar_true, D_true, Q_true in param_combinations:
    data = np.load(pathname+f'Datasets/Raw_files/skymap_data_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}.npy',
                    allow_pickle=True).item()
    m = data['m']
    metadata = data['metadata']
    NSIDE = metadata['NSIDE']
    NPIX = metadata['NPIX']
    d_vector = metadata['dipole_vector']
    q_vector_1 = metadata['quad_vector_1']
    q_vector_2 = metadata['quad_vector_2']

    dipole_quad_plotting(m, d_vector=d_vector, q_vector_1=q_vector_1, q_vector_2=q_vector_2, cmap='plasma')
    plt.savefig(pathname+f'Datasets/Skymaps/Unsmoothed/skymap_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}.png', dpi=300)
    plt.close()
    
    smooth_map = compute_smooth_map(m, angle_scale=1)

    dipole_quad_plotting(smooth_map, d_vector=d_vector, q_vector_1=q_vector_1, q_vector_2=q_vector_2, cmap='plasma')
    plt.savefig(pathname+f'Datasets/Skymaps/Smoothed/smoothed_skymap_Nbar_{N_bar_true}_D_{D_true}_Q_{Q_true}.png', dpi=300)
    plt.close()
    